Notebook for developing and testing portfolio module.

In [1]:
# import functions from utils and porteval
import sys
import os


from portopt.portfolio import Portfolio

from portopt.config import load_config
from portopt.utils import write_table

In [ ]:
# required file paths
portfolio_dir = "../data/portfolio"
config_file = os.path.join(portfolio_dir, "config.yml")
holdings_files = os.path.join(portfolio_dir, "holdings")
factor_weights_file = os.path.join(portfolio_dir, "asset_class_weights_matrix.csv")

print("portfolio_dir:", portfolio_dir)
print("config_file:", config_file)
print("factor_weights_file:", factor_weights_file)
print("holdings_files:", holdings_files)

In [ ]:
# load config
config = load_config(config_file)
import pprint
pprint.pprint(config)

In [ ]:
# define column formats for write_table function
column_formats = {
    'Ticker': {'width': 14},
    'Level_0': {'width': 14},
    'Level_1': {'width': 14},
    'Level_2': {'width': 14},
    'Level_3': {'width': 14},
    'Level_4': {'width': 14},
    'Level_5': {'width': 14},
    'Level_6': {'width': 14},
    'Factor': {'width': 24},
    'Weight': {'width': 14, 'decimal': 3, 'type':'%'},
    'Account': {'width': 25, 'align': '<'},
    'Name': {'width': 15, 'align': '<'},
    'Long Name': {'width': 15, 'align': '<'},
    'Institution': {'width': 14},
    'Type': {'width': 14},
    'Category': {'width': 14},
    'Family': {'width': 14},
    'Owner': {'width': 14},
    'Quantity': {'width': 10, 'decimal': 3},
    'Original Ticker': {'width': 14},
    'Original Quantity': {'width': 10, 'decimal': 3},
    'Original Value': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Price': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Total Value': {'width': 16, 'decimal': 2, 'prefix': '$'},
    'Allocation': {'width': 16, 'decimal': 2, 'type':'%'}
}
column_formats

In [ ]:
# create portfolio object
portfolio = Portfolio(config, factor_weights_file, holdings_files)
portfolio

In [ ]:
holdings = portfolio.getHoldings(forceRefresh=True)
write_table(holdings, columns=column_formats)

In [ ]:
tickers = portfolio.getTickers()
write_table(tickers, columns=column_formats)

In [ ]:
accounts = portfolio.getAccounts()
write_table(accounts, columns=column_formats)

In [ ]:
prices = portfolio.getPrices()
write_table(prices, columns=column_formats)

In [ ]:
factors = portfolio.getFactors()
write_table(factors, columns=column_formats)

In [ ]:
factor_weights = portfolio.getFactorWeights()
write_table(factor_weights, columns=column_formats)

In [ ]:
# Calculate allocation to Level_0 and Level_1 factors

# parameters
verbose = False
filters = None
dims = ['Level_0', 'Level_1']

# calculate metrics
metrics = portfolio.getMetrics(*dims,
                               filters=filters,
                               verbose=verbose)
write_table(metrics, columns=column_formats)

# write metrics to csv
metrics.to_csv('../exports/factor_allocations.csv')

# calculate totals
total_value = metrics['Total Value'].sum()
total_alloc = metrics['Allocation'].sum()
print(f"Total portfolio value: ${total_value:.2f}")
print(f"Total allocated value: {total_alloc:.2%}")

In [ ]:
# Calculate allocation to each ticker

# parameters
verbose = False
filters = None
dims = ['Ticker']

# calculate metrics
metrics = portfolio.getMetrics(*dims,
                               filters=filters,
                               verbose=verbose)
write_table(metrics, columns=column_formats)

# write metrics to csv
metrics.to_csv('../exports/ticker_allocations.csv')

# calculate totals
total_value = metrics['Total Value'].sum()
total_alloc = metrics['Allocation'].sum()
print(f"Total portfolio value: ${total_value:.2f}")
print(f"Total allocated value: {total_alloc:.2%}")

In [ ]:
# Calculate allocation to each account

# parameters
verbose = False
filters = None
dims = ['Account']

# calculate metrics
metrics = portfolio.getMetrics(*dims,
                               filters=filters,
                               verbose=verbose)
write_table(metrics, columns=column_formats)

# write metrics to csv
metrics.to_csv('../exports/account_allocations.csv')

# calculate totals
total_value = metrics['Total Value'].sum()
total_alloc = metrics['Allocation'].sum()
print(f"Total portfolio value: ${total_value:.2f}")
print(f"Total allocated value: {total_alloc:.2%}")

In [ ]:
# Calculate ticker allocations for a specific account

account_name = '<account_name>'

# parameters
verbose = False
filters = {'Account': [f'{account_name}']}
dims = ['Ticker']

# calculate metrics
metrics = portfolio.getMetrics(*dims,
                               filters=filters,
                               verbose=verbose)
write_table(metrics, columns=column_formats)

# write metrics to csv
metrics.to_csv(f'../exports/{account_name}_allocations.csv')

# calculate totals
total_value = metrics['Total Value'].sum()
total_alloc = metrics['Allocation'].sum()
print(f"Total portfolio value: ${total_value:.2f}")
print(f"Total allocated value: {total_alloc:.2%}")

In [ ]:
# display the ticker allocations for each account

# Get unique accounts from holdings
holdings = portfolio.getHoldings()
accounts = holdings.index.get_level_values('Account').unique()

# Iterate over accounts
for account in accounts:
    print(f"\nAllocations for {account}:")
    
    # Get metrics for this account
    metrics = portfolio.getMetrics(
        'Ticker',
        filters={'Account': [account]},
        verbose=False
    )
    
    # Display results
    write_table(metrics, columns=column_formats)
    
    # Export to CSV
    # Clean account name for filename (replace spaces and special chars with underscores)
    safe_account_name = account.replace(' ', '_').replace('(', '').replace(')', '')
    filename = f"../exports/{safe_account_name}_allocations.csv"
    metrics.to_csv(filename)
    
    # Print totals
    total_value = metrics['Total Value'].sum()
    total_alloc = metrics['Allocation'].sum()
    print(f"Total account value: ${total_value:,.2f}")
    print(f"Total allocated value: {total_alloc:.2%}")
    print(f"Results written to: {filename}")